Este script depende do script anterior **image-classification-train.py**, onde todos os imports e as definições preliminares estão alocadas.

In [ ]:
model = tfk.models.load_model(local_model)
metrics_base = ['loss', 'Accuracy', 'auc', 'precision', 'recall', 'f1_score']
metrics = history.history

# Plotar as métricas de treinamento e validação
for metric_name in metrics_base:
    local_train_values = metrics[metric_name]
    local_val_values = metrics['val_' + metric_name]  # Métrica de validação correspondente
    plt.figure(figsize=(12, 8))
    plt.plot(local_train_values, label='Train ' + metric_name.title(), lw = 0.85)
    plt.plot(local_val_values, label='Validation ' + metric_name.title(), lw = 0.85)
    plt.xlabel('Epochs')
    plt.ylabel(metric_name.title() + ' Value')
    plt.grid()
    plt.legend(edgecolor = 'k')
    plt.show()

In [ ]:
y_pred_prob = model.predict(validation_generator)
y_pred = np.argmax(y_pred_prob, axis=1)

# Obter as classes reais:
y_true = validation_generator.classes
classes = validation_generator.class_indices
cm = confusion_matrix(y_true, y_pred)

fpr, tpr, thresholds = roc_curve(y_true, y_pred)

roc_auc = auc(fpr, tpr)

# ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC curve, area = {np.round(roc_auc, 2)}', lw = 0.85)

plt.plot([0, 1], [0, 1], 'k--', label = 'Random Classifier', lw = 0.85)  # Linha diagonal para referência
plt.xlabel(r'False Positive Rate')
plt.ylabel(r'True Positive Rate')
plt.title(r'ROC Curve')
plt.grid()
plt.legend(loc='lower right', edgecolor = 'k')
plt.show()

# Confusion Matrix:
plt.figure(figsize=(8, 8))
ax = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=classes, yticklabels=classes)
ax.set_frame_on(True)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

data = {
  'Filename': validation_generator.filenames,
  'Predicted Class': [list(classes.keys())[list(classes.values()).index(pred)] for pred in y_pred]
}

df = pd.DataFrame(data)
df['Filename'] = df['Filename'].str.replace('soil/', '').str.replace('tree/', '')
df = df.sort_values(by='Filename')
df.to_csv(str(csv_dir) + f'/validation_{timestamp()}.csv', index=False)

In [ ]:
y_true, y_pred = [], []

for image_path in dataset_dir.glob("*.png"):
    # Carregando a imagem e redimensionando para o tamanho desejado
    img = tfk.preprocessing.image.load_img(image_path, target_size=image_size)

    # Convertendo a imagem para um array numpy
    img_array = tfk.preprocessing.image.img_to_array(img)

    # Pré-processamento da imagem usando o pré-processamento da VGG19
    img_array = tfk.applications.vgg19.preprocess_input(img_array)

    # Adicionando uma dimensão extra para a imagem (formato de lote)
    img_array = np.expand_dims(img_array, axis=0)

    # Fazendo a predição usando o modelo
    prediction = model.predict(img_array)

    # Convertendo a predição em um rótulo
    predicted_label = np.argmax(prediction)

    # Armazenando o rótulo verdadeiro e a predição
    y_true.append(image_path.stem)
    y_pred.append(predicted_label)

In [ ]:
data = {'Filename': y_true, 'Predicted Class': y_pred}
df = pd.DataFrame(data)

df['Predicted Class'] = df['Predicted Class'].replace({0: 'soil', 1: 'tree'})
df = df.sort_values(by='Filename')
df.to_csv(str(csv_dir) + f'/dataset_{timestamp()}.csv', index=False)

df['Predicted Class Filename'] = df['Filename'].str.split('_').str[-1].str.split('.').str[0]
df['compatibility'] = df['Predicted Class Filename'] == df['Predicted Class']

for class_name in classes.keys():
  qnt = df['Filename'].str.contains(class_name).sum()
  goalseek = (df['compatibility'] & (df['Predicted Class'] == class_name)).sum()
  perc = goalseek / qnt * 100
  print(f'{qnt} {class_name} samples.')
  print(f'{goalseek} {class_name} predicted samples.')
  print(f'{np.round(perc, 2)} % correct samples.')